<a href="https://colab.research.google.com/github/jjangmo91/ParkLab/blob/main/project/2025%20NASA%20Space%20Apps%20Challenge/03_Mekong_Delta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()
# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91') # 본인 계정

In [ ]:
# ==============================================================================
# 1단계: 분석 파라미터 설정
# ==============================================================================
# 분석 지역: 베트남 메콩 델타 최하류 (벤째(Ben Tre) 성 주변)
aoi_delta = ee.Geometry.Rectangle([105.5, 9.7, 107.0, 10.5])

# 분석 기간 설정
baseline_start = '2017-12-01'
baseline_end = '2018-03-31'
drought_start = '2019-12-01'
drought_end = '2020-03-31'

# ==============================================================================
# 2단계: 평균 SAR 이미지 생성 함수
# ==============================================================================
def create_mean_composite(start_date, end_date, aoi):
    """지정된 기간의 평균 Sentinel-1 이미지를 생성하는 함수"""
    s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq('instrumentMode', 'IW'))

    # VV와 VH 밴드를 모두 포함하는 평균 이미지 생성
    mean_image = s1_collection.select(['VV', 'VH']).mean().clip(aoi)
    return mean_image

# ==============================================================================
# 3단계: '변화 탐지' 분석 실행
# ==============================================================================
print("메콩 델타 '변화 탐지' 분석을 시작합니다...")

# 평년 및 가뭄 기간의 평균 이미지 생성
print("--> 평년 및 가뭄 기간의 평균 이미지를 생성합니다...")
baseline_image = create_mean_composite(baseline_start, baseline_end, aoi_delta)
drought_image = create_mean_composite(drought_start, drought_end, aoi_delta)

# '차이 이미지' 계산 (변화량)
# baseline_image - drought_image
# 값이 양수(+)이면: 평년보다 신호가 약해짐 (어두워짐) -> 예: 물이 마름, 식생 감소
# 값이 음수(-)이면: 평년보다 신호가 강해짐 (밝아짐) -> 예: 땅이 더 거칠어짐
difference_image = baseline_image.subtract(drought_image)
print("--> '차이 이미지' 계산 완료.")

# ==============================================================================
# 4단계: 지도 시각화
# ==============================================================================
# 시각화 파라미터
vis_params_rgb = {'min': [-20, -25, -5], 'max': [0, -5, 5], 'bands': ['VV', 'VH', 'VV']}
vis_params_diff = {'min': -3, 'max': 3, 'palette': ['blue', 'white', 'red']} # 파랑:밝아짐, 흰색:변화없음, 빨강:어두워짐

# 결과 지도 생성
m_delta = geemap.Map(center=[10.1, 106.2], zoom=9)
m_delta.add_basemap('HYBRID')

# 원본 이미지들 추가 (처음에는 꺼진 상태로)
m_delta.addLayer(baseline_image, vis_params_rgb, '평년(2018) 모습', False)
m_delta.addLayer(drought_image, vis_params_rgb, '가뭄(2020) 모습', False)

# 핵심 결과인 '차이 이미지' 추가
m_delta.addLayer(difference_image.select('VV'), vis_params_diff, '변화 탐지 결과 (VV 편파)')

# 범례 추가
m_delta.add_legend(
    title="SAR 신호 변화 (평년-가뭄)",
    legend_dict={
        '신호 강해짐 (밝아짐)': '0000FF',
        '변화 거의 없음': 'FFFFFF',
        '신호 약해짐 (어두워짐)': 'FF0000'
    },
)

m_delta.addLayerControl()
display(m_delta)

붉은색 지역은 VV + VH 모두 감소: 경작이 중단되고 땅이 말라버렸다는 가설
vv 감소: 벼가 있던 땅이 농사 안지으면서 매끄러운 마른 땅으로 변하면서
vh 감소: 벼와 같은 식생의 volume 사라지면서, 복잡한 산란을 일으킬 구조물이 없어져 신호가 약해짐

파란색 지역은 VV + VH 모두 증가: 단순한 논에서 복잡한 구조의 양식장으로 바뀌었다??
vv 증가: 난반사 증가하여 신호가 강해짐
vh 증가: 신호가 여러번 부딪히는 이중 반사가 발생하여, 복잡한 산란 신호인 vh가 강해짐